# Main-and-Transfer Substation Test

## Create a New Substation

### Import CIMantic Graphs Library and CIM profile

In [8]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters, RDFlibConnection, BlazegraphConnection
import cimgraph.utils as utils

import importlib

cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

params = ConnectionParameters(filename=None, cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

### Create New Main-and-Transfer Substation

In [9]:
from cimbuilder.substation_builder import MainAndTransferSubstation

In [10]:
SubBuilder = MainAndTransferSubstation(connection=connection, name="main_transfer_sub", base_voltage = 115000)
substation = SubBuilder.substation
print(substation)

Substation(mRID='0796763b-f850-4d9c-baee-2713f92f92ef', aliasName=None, description=None, name='main_transfer_sub', Names=[], AssetDatasheet=None, Assets=[], ConfigurationEvent=[], Controls=[], Location=None, Measurements=[], OperatingShare=[], PSRType=None, ReportingGroup=[], ConnectivityNodes=[], TopologicalNode=[], AdditionalGroupedEquipment=[], Equipments=[], Bays=[], NamingFeeder=None, NormalEnergizedFeeder=[], NormalEnergizingFeeder=[], Region=None, VoltageLevels=[])


### Add IEEE 13 Feeder and IEEE 13 Assets Feeder to substation

In [11]:
# Import 13 bus model from XML file
ieee13_feeder = cim.Feeder(mRID = '49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
params = ConnectionParameters(filename='../test_models/IEEE13.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
ieee13_network = FeederModel(connection=connection, container=ieee13_feeder, distributed=False)


In [12]:

assets13_feeder = cim.Feeder(mRID = '5B816B93-7A5F-B64C-8460-47C17D6E4B0F')
params = ConnectionParameters(filename='../test_models/IEEE13_Assets.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
assets13_network = FeederModel(connection=connection, container=assets13_feeder, distributed=False)


In [13]:
SubBuilder.new_feeder(series_number = 10, feeder=ieee13_feeder, feeder_network=ieee13_network)
SubBuilder.new_feeder(series_number = 20, feeder=assets13_feeder, feeder_network=assets13_network)

In [14]:
SubBuilder.network.pprint(cim.Substation)
SubBuilder.network.pprint(cim.Feeder)

{
    "0796763b-f850-4d9c-baee-2713f92f92ef": {
        "mRID": "0796763b-f850-4d9c-baee-2713f92f92ef",
        "name": "main_transfer_sub"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "NormalEnergizingSubstation": "0796763b-f850-4d9c-baee-2713f92f92ef"
    },
    "5B816B93-7A5F-B64C-8460-47C17D6E4B0F": {
        "mRID": "5B816B93-7A5F-B64C-8460-47C17D6E4B0F",
        "NormalEnergizingSubstation": "0796763b-f850-4d9c-baee-2713f92f92ef"
    }
}


### Save Substation to XML File

In [15]:
utils.write_xml(SubBuilder.network, '../test_output/main_and_transfer.xml')

![main-and-transfer](./main_transfer.png)

## Round-Trip Test: Load and Read from Database

### Delete all old entries from database and load new models

In [16]:
# Connect to Blazegraph Database
from cimgraph.databases import BlazegraphConnection
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
blazegraph = BlazegraphConnection(params)
blazegraph.execute('drop all')

/home/singha42/.cache/pypoetry/virtualenvs/cim-builder-hXthbBAa-py3.10/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=43ms, elapsed=43ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=55ms, commitTime=1708969133304, mutationCount=6538</p\n></html\n>'

In [17]:
from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
loader = BlazegraphLoader(params)

In [18]:
loader.upload_from_file(filename='../test_models/IEEE13.xml')
loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
loader.upload_from_file(filename='../test_output/main_and_transfer.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="39"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="42"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="278" milliseconds="20"/>

### Create new NodeBreakerModel of Substation + Feeders

In [19]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [20]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "0796763b-f850-4d9c-baee-2713f92f92ef": {
        "mRID": "0796763b-f850-4d9c-baee-2713f92f92ef",
        "name": "main_transfer_sub",
        "ConnectivityNodes": "['b2fb9395-08cc-42fc-80c8-951d16e17261', 'c523f35b-862b-4058-970e-d3a137378d95', 'c78c29fe-a44b-4ef3-a4db-c952ace743f0', 'ee5bdf91-2ae6-4ee8-b8f2-40a8dbf31a80', 'f11af68f-098c-4bd0-bbcc-0258bd27216d', 'fa5dc9ff-dff8-45fe-ae34-a8e3ea97589e']",
        "Equipments": "['9fe3a69b-cdf1-4d1d-8d66-f6614a856f3f', 'cf49ad55-c6f6-4a02-969b-02aca93ff28d', 'cfabbb9c-47c0-4aba-8ab5-f3a2f655cad6', 'ea766b62-b218-4297-be25-6b0badc6aa9c', 'f3a55d6a-f712-4dcb-8302-87a13be0bb5a', 'f8077142-7834-42e5-98df-625d1492f5fb', 'fa807bc9-76f3-47f3-b82f-184251c48e3e', '0575060d-7134-406f-8acc-472ebb99ce8a', '1084a0be-e761-449f-8462-74daf0d30a33', '4cbb5db3-f363-46a9-93f4-894950a5df83', '509342b1-f0d2-44e8-8d6d-57b844968282', '51b98695-288e-496d-a5ff-ead76ec54efc', '599d4234-f812-4dc0-830c-1107cd67d265']",
        "NormalEnergizedFeeder": "['49AD

In [21]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', '0F1E28C3-6C44-4F88-B79C-2FDBCA4487B2', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '421E99BE-A834-4809-B924-84D88F634A45', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '673E896A-DCBF-4E43-9924-BEB31C5B6005', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '76D6D03C-967C-4E12-836D-A98DF3552BC7', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '8C58660F-C62C-4903-BE72-22F1255B1E62', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', '94F822E0-7130-4205-8597-B47110BBEF4B', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', 'DBFA8EFA-B397-40DA-9642-60E8DE4B3471', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '0124E881-B8

In [22]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


### Merge substation and feeders into a single XML file 

In [23]:
utils.get_all_data(network)

ConnectivityNode
Terminal
Disconnector
BusbarSection
Breaker
Substation
Feeder
TransformerTank
ACLineSegment
PowerTransformer
LoadBreakSwitch
EnergyConsumer
Fuse
PowerElectronicsConnection
EnergySource
LinearShuntCompensator
Recloser
Location
BaseVoltage
LoadResponseCharacteristic
EnergyConsumerPhase
OperationalLimitSet
TopologicalNode
TapChangerControl
TransformerTankEnd
PowerTransformerEnd
RegulatingControl
WireSpacingInfo
ACLineSegmentPhase
PerLengthPhaseImpedance
OverheadWireInfo
ConcentricNeutralCableInfo
TapeShieldCableInfo
PhaseImpedanceData
WirePosition
TransformerTankInfo
RatioTapChanger
TransformerEndInfo
ShortCircuitTest
NoLoadTest
TransformerMeshImpedance
TransformerCoreAdmittance
BatteryUnit
PowerElectronicsConnectionPhase
PhotovoltaicUnit
CurrentLimit
VoltageLimit
OperationalLimitType
CoordinateSystem
PositionPoint
SubGeographicalRegion
SwitchPhase
LinearShuntCompensatorPhase
TopologicalIsland
GeographicalRegion


In [24]:
utils.write_xml(network, '../test_output/main_transfer_and_feeders.xml')
